In [1]:
pip install pdfplumber numpy openai redis tabulate


[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
# Cell 1: Import Libraries and Setup
import pdfplumber
import fitz  # PyMuPDF
from langchain.text_splitter import RecursiveCharacterTextSplitter
import pandas as pd
from openai import OpenAI
from redis import Redis
from redis.commands.search.field import VectorField, TextField
import numpy as np
import os

# OpenAI API setup
API_KEY = "sk-1b0e5d114e3b4bb1ac9dbef07a531b10"  # Replace with your actual API key
client = OpenAI(api_key=API_KEY, base_url="https://dashscope.aliyuncs.com/compatible-mode/v1")

# Redis setup
r = Redis()  # Assumes Redis is running on default host and port
INDEX_NAME = "PDFData"
VECTOR_DIM = 1024
DISTANCE_METRIC = "COSINE"

pdf_path = "ICBC_2024_FYR.pdf"


In [5]:
# Cell 2: PDF Content Extraction
def extract_text_from_pdf(pdf_path):
    with pdfplumber.open(pdf_path) as pdf:
        text = "".join(page.extract_text() or "" for page in pdf.pages)
    return text

def extract_tables_from_pdf(pdf_path):
    tables = []
    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            page_tables = page.extract_tables()
            tables.extend(page_tables)
    return tables

def extract_images_from_pdf(pdf_path):
    pdf_document = fitz.open(pdf_path)
    images = []
    for page_num in range(len(pdf_document)):
        page = pdf_document[page_num]
        image_list = page.get_images(full=True)
        for img in image_list:
            xref = img[0]
            base_image = pdf_document.extract_image(xref)
            images.append(base_image["image"])
    return images

# Extract content
text = extract_text_from_pdf(pdf_path)
tables = extract_tables_from_pdf(pdf_path)
images = extract_images_from_pdf(pdf_path)
for i, img_data in enumerate(images):
    with open(f"image_{i}.png", "wb") as f:
        f.write(img_data)

print(f"Extracted text length: {len(text)} characters")
print(f"Extracted {len(tables)} tables")
print(f"Extracted {len(images)} images")


CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, def

Extracted text length: 338123 characters
Extracted 560 tables
Extracted 18 images


In [6]:
# Cell 3: Document Splitting
def split_text(text, chunk_size=1000, chunk_overlap=200):
    splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
        separators=["\n\n", "\n", " ", ""]
    )
    return splitter.split_text(text)

def table_to_markdown(table):
    if not table or not table[0]:
        return ""
    df = pd.DataFrame(table[1:], columns=table[0])
    return df.to_markdown(index=False)

# Split text and convert tables
text_chunks = split_text(text)
table_chunks = [table_to_markdown(table) for table in tables]
# 打印前20个文本块和5个表格块
print("First 20 text chunks:")
for chunk in text_chunks[:20]:
    print(chunk[:500] + "...")
print("\nFirst 5 table chunks:")
for chunk in table_chunks[:5]:
    print(chunk[:500] + "...")

# 为所有保存的图片生成描述
image_descriptions = []
import base64
#  base 64 编码格式
def encode_image(image_path):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode("utf-8")
    
for i in range(len(images)):
    image_path = f"image_{i}.png"  # 与之前保存的图片路径一致
    base64_image = encode_image(image_path)
    try:
        response = client.chat.completions.create(
            model="qwen2.5-vl-32b-instruct",
            messages=[
                {"role": "user", "content": [
                    {"type": "text", "text": "提取图片中的信息，需要精准描述，不要漏掉信息，但是也不需要额外解释。若图片为饼状图、折现图和柱状图等，请使用饼状图、折现图和柱状图等关键词，并以 json 格式返回。若图片为表格，请使用表格的关键词，并以 markdown 格式返回。若图片为架构图、流程图等，请使用架构图、流程图等关键词，并以 mermaid 格式返回。"},
                    {"type": "image_url", "image_url": {"url": f"data:image/png;base64,{base64_image}"}}
            ],}]
        )
        description = response.choices[0].message.content
        image_descriptions.append(description)
        print(f"Generated description for image {i}: {description}")
    except Exception as e:
        print(f"Error processing image {i}: {str(e)}")
        image_descriptions.append("")
print(f"Split text into {len(text_chunks)} chunks")
print(f"Converted {len(table_chunks)} tables to Markdown")
print(f"Generated {len(image_descriptions)} image descriptions")

First 20 text chunks:
中国工商银行股份有限公司
（股票代码：601398）
2024 年度报告公司简介
中国工商银行成立于 1984年1月1日。2005年10月28日，本行整体改制
为股份有限公司。2006 年 10 月 27 日，本行成功在上交所和香港联交所同日挂
牌上市。
本行致力于建设中国特色世界一流现代金融机构，拥有优质的客户基础、多
元的业务结构、强劲的创新能力和市场竞争力。本行将服务作为立行之本，坚持
以服务创造价值，向全球超1,300万对公客户和 7.66亿个人客户提供丰富的金融
产品和优质的金融服务，以自身高质量发展服务经济社会高质量发展。本行自觉
将社会责任融入发展战略和经营管理活动，在服务制造业、发展普惠金融、支持
乡村振兴、发展绿色金融、支持公益事业等方面受到广泛赞誉。
本行始终聚焦主业，坚持服务实体经济的本源，与实体经济共荣共存、共担
风雨、共同成长；始终坚持风险为本，牢牢守住底线，不断提高控制和化解风险
的能力；始终坚持对商业银行经营规律的把握与遵循，致力于成为基业长青的银
行；始终坚持稳中求进、创新求进，持续深化重点发展战略，积极发展金融科技，
加快数字化转型；始终坚持专业专注，...
动能、多元化结构、生态化基础等“五化”转型。
中国工商银行股份有限公司2024年度报告（A股） 2使 命
提供卓越金融服务
服务客户 回报股东 成就员工 奉献社会
愿 景
全面建设具有中国特色的世界一流现代金融企业，
成为基业长青的银行
价值观
工于至诚 行以致远
诚信 人本 稳健 创新 卓越
中国工商银行股份有限公司2024年度报告（A股） 3目录
1. 释义 ........................................................................................................................ 5
2. 2024 年主要排名与奖项 ....................................................................................... 7
3. 重要提示 .....................................................

In [24]:
from tqdm.notebook import tqdm  # Jupyter专用进度条

# Cell 4: Vectorization (with progress bars)
def get_embedding(text, model="text-embedding-v3"):
    response = client.embeddings.create(
        input=text[:8192*2],  # Truncate to approximate token limit
        model=model,
        dimensions=VECTOR_DIM
    )
    return response.data[0].embedding

# Generate embeddings with progress bars
print("Generating text embeddings...")
text_embeddings = [get_embedding(chunk) for chunk in tqdm(text_chunks, desc="Text chunks")]

print("\nGenerating table embeddings...")
table_embeddings = [get_embedding(chunk) for chunk in tqdm([c for c in table_chunks if c], desc="Tables")]

print("\nGenerating image embeddings...")
image_embeddings = [get_embedding(desc) for desc in tqdm(image_descriptions, desc="Images")]

print(f"\nGenerated {len(text_embeddings)} text embeddings")
print(f"Generated {len(table_embeddings)} table embeddings")
print(f"Generated {len(image_embeddings)} image embeddings")

Generating text embeddings...


Text chunks:   0%|          | 0/423 [00:00<?, ?it/s]


Generating table embeddings...


Tables:   0%|          | 0/560 [00:00<?, ?it/s]


Generating image embeddings...


Images:   0%|          | 0/18 [00:00<?, ?it/s]


Generated 423 text embeddings
Generated 560 table embeddings
Generated 18 image embeddings


In [25]:
# Cell 5: Data Storage in Redis
# Define index schema
md_embedding_field = VectorField(
    "md_embedding", "FLAT",
    {"TYPE": "FLOAT32", "DIM": VECTOR_DIM, "DISTANCE_METRIC": DISTANCE_METRIC}
)
content_field = TextField("content")
type_field = TextField("type")
fields_for_index = [content_field, type_field, md_embedding_field]

# Create index
try:
    r.ft(INDEX_NAME).create_index(fields=fields_for_index)
    print(f"Index '{INDEX_NAME}' created successfully")
except Exception as e:
    print(f"Index creation failed: {e}")

# Store data
def store_data(chunks, embeddings, data_type):
    for i, (chunk, embedding) in enumerate(zip(chunks, embeddings)):
        key = f"{INDEX_NAME}:{data_type}:{i}"
        mapping_data = {
            "md_embedding": np.array(embedding, dtype=np.float32).tobytes(),
            "content": chunk,
            "type": data_type
        }
        r.hset(key, mapping=mapping_data)

store_data(text_chunks, text_embeddings, "text")
store_data(table_chunks, table_embeddings, "table")
store_data(image_descriptions, image_embeddings, "image")

print("Data stored in Redis successfully")

Index 'PDFData' created successfully
Data stored in Redis successfully


In [31]:
import numpy as np
import json
from redis.commands.search.query import Query
# Search
# user_question = "工商银行2024年海外布局？"
user_question = "工商银行为什么被成为”宇宙行“？"

# Helper functions
def json_gpt(input: str):
    completion = client.chat.completions.create(
        model="qwen2.5-7b-instruct",
        messages=[
            {"role": "system", "content": "Output only valid JSON"},
            {"role": "user", "content": input},
        ],
        temperature=0.2,
    )

    text = completion.choices[0].message.content
    parsed = json.loads(text)

    return parsed


HA_INPUT = f"""
You are a financial report analysis assistant.
You have access to a search API that returns relevant sections from a financial report.
Generate a search query by extracting key words from the user's question.

User question: {user_question}

Format: {{"searchQuery": "search query"}}
"""
query_str = json_gpt(HA_INPUT)["searchQuery"]
print(query_str)

query_embedding = client.embeddings.create(input=query_str, model="text-embedding-v3", dimensions=1024, encoding_format="float")
query_vec = np.array(query_embedding.data[0].embedding, dtype=np.float32).tobytes()
# Prepare the query
k_nearest = 3  # Retrieve top 3 relevant chunks
query_base = (Query(f"*=>[KNN {k_nearest} @md_embedding $vec as score]").sort_by("score").return_fields("score", "content", "type").dialect(2))
query_param = {"vec": query_vec}
try:
    query_results = r.ft(INDEX_NAME).search(query_base, query_param).docs
    print(f"\nRetrieved {len(query_results)} results:")
    for i, doc in enumerate(query_results):
        print(f"\n--- Result {i+1} ---")
        print(f"Type: {doc.type}")
        print(f"Score: {doc.score}")
        print(f"Content (first 200 chars): {doc.content[:200]}...")
except Exception as e:
    print(f"Error executing vector query: {e}")
    query_results = []
# Prepare context for the LLM
context = "\n\n".join([doc.content for doc in query_results])

工商银行 宇宙行

Retrieved 3 results:

--- Result 1 ---
Type: text
Score: 0.391143679619
Content (first 200 chars): 电话：0755-82246400 传真：0571-87808207
传真：0755-82246247
四川分行 工银瑞信基金管理有限公司
地址：四川省成都市锦江区总府路45号 地址：北京市西城区金融大街5号新盛大厦
邮编：610020 A座
电话：028-82866000 邮编：100033
传真：028-82866025 电话：010-66583349
传真：010-66583158
天津分行
...

--- Result 2 ---
Type: text
Score: 0.393015921116
Content (first 200 chars): 有限公司
工银国际控股有限公司 投资银行 59.63亿港元 7,016.21 1,058.98 2.70
中国工商银行（澳门）
商业银行 5.89亿澳门元 46,577.07 3,913.78 18.47
股份有限公司
中国工商银行（印度尼 3.71万亿
商业银行 3,147.68 449.06 29.07
西亚）有限公司 印尼盾
中国工商银行马来西亚
商业银行 8.33亿林吉特 1,190.99...

--- Result 3 ---
Type: text
Score: 0.39324349165
Content (first 200 chars): 已发行股本/
股权比例% 实收资本面值
2024年 2023年 2024年
公司名称 12月31日 12月31日 12月31日 本行投资额 注册地及成立日期 业务性质
通过设立或投资等方式
取得的主要子公司：
中国工商银行马来西 马来西亚吉隆坡
亚有限公司 100 100 8.33亿林吉特 8.33亿林吉特 2010年1月28日 商业银行
中国工商银行（阿拉木 哈萨克斯坦阿拉木图
图）股份公司 1...


In [32]:
system_prompt = "你是一个金融报告分析专家。请根据搜索结果回答用户提问，注意，请务必首先依赖搜索结果，而不是你自己已有的知识。如果搜索结果不包含能够回答用户提问的信息，你可以说“抱歉，我无法回答这个问题”。"
messages = [{"role": "system", "content": system_prompt},
            {"role": "user", "content": "用户提问：" + user_question},
            {"role": "user", "content": "搜索结果：" + context}]
response = client.chat.completions.create(
                    messages=messages,
                    model="qwen2.5-32b-instruct", #"qwen-max",
                    max_tokens=1000
                )
print(response.choices[0].message.content)

抱歉，我无法回答这个问题。提供的搜索结果中没有包含关于工商银行被称为"宇宙行"的原因的信息。
